<table style="width: 100%; border-style: none;">
<tr style="border-style: none">
<td style="border-style: none; width: 1%; font-size: 16px">Institut f&uuml;r Theoretische Physik<br /> Universit&auml;t zu K&ouml;ln</td>
<td style="border-style: none; width: 1%; font-size: 16px">&nbsp;</td>
<td style="border-style: none; width: 1%; text-align: right; font-size: 16px">Prof. Dr. Simon Trebst<br />Peter Br&ouml;cker</td>
</tr>
</table>
<hr>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">Computerphysik</h1>
<h1 style="font-weight:bold; text-align: center; margin: 0px; padding:0px;">&Uuml;bungsblatt 5</h1>
<hr>
<h3 style="font-weight:bold; text-align: center; margin: 0px; padding:0px; margin-bottom: 20px;">Sommersemester 2016</h3>

**Website:** [http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml](http://www.thp.uni-koeln.de/trebst/Lectures/2016-CompPhys.shtml)

**Abgabe**: Montag, 23. Mai, 2016 vor der Vorlesung

**Name**: <font color="red">Bitte geben Sie ihren Namen an</font>

**Matrikelnummer**: <font color="red">Bitte geben Sie ihre Matrikelnummer an</font>

<table style="border-style: none; width: 100%"><tr style="border-style: none;">
<td style="border-style: none; width: 1%; font-size: 20px; font-weight: bold">Monde auf der Überholspur</td>
<td style="border-style: none; width: 1%; text-align: right">(5 Punkte)</td></tr></table>

Die Monde **Janus** und **Epimetheus** umkreisen den Planeten Saturn *koorbital*, d.h. auf nahezu der gleichen Bahn.
Bemerkenswerterweise ist dabei der Bahnunterschied zwischen dem inneren und dem äußeren der beiden Monde deutlich
kleiner als deren Durchmesser. Alle vier Jahre begegnen sich die beiden Monde, da der innere Mond ein wenig schneller unterwegs ist als der Äußere. Wieso es dabei nicht zu einer Kollision kommt, wollen wir in dieser Aufgabe untersuchen.

Dazu wollen wir das **3-Körper-System** aus Saturn, Janus und  Epimetheus simulieren. Für die Simulation bewegen wir uns ins Bezugssystem des Saturn. Zwischen zwei Körpern wirkt dann eine Gravitationskraft gemäß

\begin{equation}
\mathbf{F}_{12}=-G\frac{m_1 m_2}{|\mathbf{r}_1-\mathbf{r}_2|^3}\left(\mathbf{r}_1-\mathbf{r}_2\right),
\end{equation}

wobei wir die Gravitationskonstante $G = 1$ setzen (wodurch sich die Größen der Parameter natürlich von den realistischen unterscheiden). Berechnen Sie die Bahnen der Monde mittels des **Verlet**-Verfahrens und plotten Sie die Bahnkurven der Monde. Verwenden Sie folgende Startwerte:

\begin{align*}
m_1 &= 1 \\
m_2 &= 4 \\
m_{\text{Saturn}} &= 4\times10^4 \\
\mathbf{r}_1 &= (-155, 0) \quad \frac{d\mathbf{r}_1}{dt} = (0, -16.1) \\
\mathbf{r}_2 &= (150, 0), \quad \frac{d\mathbf{r}_2}{dt} = (0, 16.3)
\end{align*}

### Bemerkungen

* berechnet wird die Bewegung zweier Monde der Massen $m_1$ und $m_2$ um ein Zentrum der Masse $M$
* der Zentralkörper (hier: Saturn) ruht im Ursprung des Koordinatensystems
* die *reduzierte Masse* $\mu$ muss nicht berechnet werden, da $M\gg m_1, m_2$
* die Formel für die Verlet-Methode lautet: $f_{i+1} = 2\cdot f_{i} - f_{i-1} + a*\Delta t^2$
* die Werte für die Beschleunigungen $a_i(t)$ sind für die Berechnung notwendig, aber uninteressant

In [29]:
function verlet_2_moons(t_0, t_end, dt, m1, m2, M, r1_0, r2_0, v1_0, v2_0, save_vs=false, return_as=false, G=6.67408e-11)
    # initialize variables: arrays (r, v, a)
    # attention: length(v) == length(r)-2 == length(a)-1
    times = collect(t_0:dt:t_end)
    body1 = tuple(zeros(length(times)), zeros(length(times)), zeros(length(times))) # (r, v, a)
    body2 = tuple(zeros(length(times)), zeros(length(times)), zeros(length(times))) # (r, v, a)
    a(r1, r2, m) = -G*M/abs(r2-r1)^3*(r2-r1) # a_i = F/m_i
    
    # t=t0
    #   save r0
    println(body1[1][1])
    body1[1][1] = r1_0
    body2[1][1] = r2_0
    #   calc a
    body1[1][3] = a(body1[1][1], body1[1][2], m1)
    body2[1][3] = a(body1[1][1], body1[1][2], m2)
    # t=t0+dt: calculate r, a
    # for all t from t0+2dt to t_end: calculate r, a
        # use verlet 
    # if save_velocities 
        # for all t from t0+dt to t_end-dt
            # calculate v

    # return everything wanted
#    if return_as
        #saves_vs == true ? (return (body1, body2)):(return (body1[1:2:3], body2[1:2:3]))
#    else
#        saves_vs == true ? (return (body1[1:2], body2[1:2])):(return (body1[1], body2[1]))
#    end
end

t0 = 0
t1 = 100
dt = 1e-3
verlet_2_moons(t0, t1, dt, 1, 4, 4e4, [-155, 0], [150, 0], [0, -16.1], [0, 16.3])
;

0.0


LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Float64}, ::Array{Int32,1})
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Float64}, !Matched::Int8)
  convert(::Type{Float64}, !Matched::Int16)
  ...
while loading In[29], in expression starting on line 35